In [ ]:
%%bash
pip install astroML
pip install sklearn

In [1]:
import numpy as np
import probability_distributions

import simple_set_functions
from probability_distributions import ProbabilityDict
from probability_distributions import JointProbabilityMatrixExtended
from jointpdf.jointpdf import JointProbabilityMatrix
from jointpdf.jointpdf import FullNestedArrayOfProbabilities

In [2]:
from functools import reduce

marginal_distributions = np.array([[0.5, 0.5], [0.5, 0.5], [0.5, 0.5], [0.5,  0.5]])

def compute_joint(multi_index, marginal_distributions):
    return reduce(lambda x, y: x*y, [marginal_distributions[combination] for
            combination in zip(range(len(multi_index)), multi_index)])

def compute_joint_from_marginals(marginal_distributions):
    """compute full joint probability matrix given independent marginals
    
    Parameters:
    ----------
    marginal_distributions: numpy array 
        A 2 dimensional array where the first axis are the variables
        and the second axis the number of states
        
    Returns: 
    -------
    A numpy array where every dimension is one variable, it can be seen as 
    a tree build from the variables 
    """
    dimension_joint_distribution = [len(dist) for dist in marginal_distributions]
    full_distribution_temp = np.zeros(dimension_joint_distribution)
    full_distribution = np.zeros(dimension_joint_distribution)

    it = np.nditer(full_distribution, flags=['multi_index'], op_flags=[['readwrite']])
    while not it.finished:
        full_distribution[it.multi_index]= compute_joint(it.multi_index, marginal_distributions)
        it.iternext()

    return full_distribution

def create_uniform_marginal_distributions(num_states, num_variables):
    """create a list of uniform probability distributions 
    
    params:
        num_states: how many states the variables have
        num_variables: how many variables to create
    """
    
    return np.array([[1.0/num_states]*num_states]*num_variables)
    
#print(compute_joint_from_marginals(create_uniform_marginal_distributions(3, 3)))




Tryout whether the implementation works

In [3]:
pdf = JointProbabilityMatrix(4, 2, compute_joint_from_marginals(marginal_distributions))
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)
pdf_extended.adjust_to_old_format()

marginal_extended = pdf_extended.marginalize_distribution([0, 1])
marginal = pdf.marginalize_distribution([0, 1])

pdf_extended.append_determinstic_function(simple_set_functions.double_xor, [2, 2])
pdf_extended.adjust_to_old_format()
marginal_extended = pdf_extended.marginalize_distribution([0, 1])

print(np.all(
        marginal_extended.joint_probabilities.joint_probabilities==marginal.joint_probabilities.joint_probabilities
))

probability_array = np.array(
    [
        [
            [0.1, 0.2, 0.25]
        ],
        [
            [0.2, 0.1, 0.15]
        ]
    ]
)

pdf_extended = JointProbabilityMatrixExtended(
    3, 3, FullNestedArrayOfProbabilities(probability_array)
)
first_dict = ProbabilityDict(pdf_extended.to_dict())
pdf_extended.adjust_to_old_format()
second_dict = ProbabilityDict(pdf_extended.to_dict())

print("first dict")
first_dict.print_distribution(True)
print("second dict")
second_dict.print_distribution(True)

True
first dict
(0, 0, 0): 0.1
(0, 0, 1): 0.2
(0, 0, 2): 0.25
(1, 0, 0): 0.2
(1, 0, 1): 0.1
(1, 0, 2): 0.15
second dict
(0, 0, 0): 0.1
(0, 0, 1): 0.2
(0, 0, 2): 0.25
(1, 0, 0): 0.2
(1, 0, 1): 0.1
(1, 0, 2): 0.15


In [ ]:
import itertools

def calculate_mi_profile(joint_distribution, function_labels, testing=False):
    """create a mutual information profile
    
    params:
        joint_distribution: A JointProbabilityMatrix object from the jointpdf package.
        The joint probabilities properties should be of both all the variables and the
        function.
        function_labels: The labels in the joint distribution which the function represents
        
    returns: a 1-d nd-array where every value represents the average normalized mutual information
        between subsets of variables (of the size of the index) and the ouput variable. 
    """

    number_of_arguments = joint_distribution.numvariables-len(function_labels)
    if testing:
        print("the number of arguments is {}".format(number_of_arguments))
    mi_values = np.zeros(number_of_arguments+1)
    for number_of_variables in np.arange(1, number_of_arguments+1, 1):
        combinations = itertools.combinations(range(number_of_arguments), 
                                              number_of_variables)
        
        mi_values[number_of_variables] = np.mean(
            [joint_distribution.mutual_information(list(combination), function_labels) 
             for combination in combinations]
        )
        
        if testing:
            for comb in itertools.combinations(range(number_of_arguments), number_of_variables):
                print("input variables {} function labels {} mi value {}".format(
                    list(comb), 
                    function_labels,
                    joint_distribution.mutual_information(list(comb), function_labels)
                ))
        
    return mi_values
   
def create_mutual_information_profile(pdf, function, dim_output_function, 
                                      num_states_output_function, testing=False):
    """create mutual information profile values
    
    params:
        pdf: An object of the JointProbabilityMatrixExtendedClass
        function: a function object that takes an iterable of the same length
        as the number of variables in the pdf_extended object
        dim_output_function: the dimension (number of variables) the function outputs
        num_states_output_function: a list, where every entry represents the amount of 
        states that variable has. The length should be equal to dim_output_function
        
    returns: An array representing the MI profile values for number of variables 
    starting at zero and going to the total amount of variables in the distribution
    """

    pdf.append_determinstic_function(double_xor, dim_output_function, num_states_output_function)
    pdf.adjust_to_old_format()
    
    function_indices = list(range(pdf.numvariables-dim_output_function, pdf.numvariables))
    
    if testing:
        print("the function indices are {}".format(function_indices))
    
    return calculate_mi_profile(pdf_extended, function_indices)
    

Do some testing. I found out that xor for i.i.d variables has mutual information with the individual variables.
This was a huge surprise so I tested it with multiple implementation and also using playground.ipnb. 

In [ ]:
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)

pdf_extended.append_determinstic_function(double_xor, num_variables=2, num_states=[2, 2])
pdf_extended.adjust_to_old_format()
print(calculate_mi_profile(pdf_extended, [4, 5]))

print("now using the create function")
print("")


pdf_extended2 = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)

print(create_mutual_information_profile(pdf_extended2, double_xor, 2, [2,2]))

In [ ]:
print("first examining the double xor function on uniform distribution")
pdf_extended3 = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions[:4]))
)
print(create_mutual_information_profile(pdf_extended3, double_xor, 2, [2,2]))

print("now examining the subset and function on uniform distribution")
num_states1 = 2
num_variables1 = 4
probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(create_uniform_marginal_distributions(2, 4))
)
pdf_extended4 = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
subset_and_func = subset_and([(0,2), (1,3)])  
print(create_mutual_information_profile(pdf_extended4, subset_and_func, 2, [2,2]))

print("now examining the sum function")
num_states1 = 2
num_variables1 = 4
probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(create_uniform_marginal_distributions(2, 4))
)
pdf_extended5 = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
pdf_extended5.append_determinstic_function(sum, 1, [5])

probabability_dict5 = pdf_extended5.to_dict()
#ProbabilityDict(probabability_dict5).print_dict(True)

pdf_extended5.adjust_to_old_format()
probabability_dict5 = ProbabilityDict(pdf_extended5.to_dict())
print("max entropy is: {}".format(probabability_dict5.calculate_entropy([4])))
print("the synergy in the distribution is {}".format(
    sum([probabability_dict5.calulate_mutual_information([i], [4]) for i in range(4)])
))
#print(pdf_extended5.joint_probabilities.joint_probabilities.shape)
print(calculate_mi_profile(pdf_extended5, [4]))


num_states1 = 2
num_variables1 = 2
marginal_distributions = np.array(
    [
        [0.3, 0.7],
        [0.5, 0.5],
        [0.6, 0.4],
        [0.6, 0.4]
    ]
)


print("now examing the copy function")

probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(marginal_distributions[2:])
)
pdf_extended6 = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
pdf_extended6.append_determinstic_function(copy, 2, [2, 2])
pdf_extended6.adjust_to_old_format()
prob_dict = ProbabilityDict(pdf_extended6.to_dict())
prob_dict.print_dict()
print(calculate_mi_profile(pdf_extended6, [2, 3], testing=False))

print("now for the xor function")

probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(marginal_distributions[2:])
)
pdf_extended_xor = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
pdf_extended_xor.append_determinstic_function(xor, 1, [2])
pdf_extended_xor.adjust_to_old_format()
prob_dict = ProbabilityDict(pdf_extended_xor.to_dict())
prob_dict.print_dict()
print("the synergy in the distribution is {}".format(
    sum([prob_dict.calulate_mutual_information([i], [2]) for i in range(2)])
))

print(calculate_mi_profile(pdf_extended_xor, [2], testing=False))



Thought: already information about function for zero variables, how to deal with that? Entropy in function would make sense but how to use it I do not know. How far is entropy in function from entropy in totally uniform distributed function? Seems like a reasonable idea, for the zeroth level mi-profile value

Add function to go from MI-Profile to information levels

In [ ]:
import math

def choose(n, m):
    if m > n or m < 0 or n < 0:
        raise ValueError("please provide integers and n bigger equal to m")
        
    return math.factorial(n)/(math.factorial(m)*math.factorial(n-m))   

mutual_info_profile = [0, 0, 1.0/3.0, 1, 2]
mutual_info_profile = mutual_info_profile[1:]
number_of_variables = len(mutual_info_profile)

first_level = choose(number_of_variables, 1) * mutual_info_profile[0] 
second_level = (choose(number_of_variables, 2) * mutual_info_profile[1] - 
                choose(number_of_variables, 2) * choose(2,1) / choose(number_of_variables, 1) * first_level)

third_level = (choose(number_of_variables, 3) * mutual_info_profile[2] - 
               choose(number_of_variables, 3) * choose(3, 2) / choose(number_of_variables, 2) * second_level -
               choose(number_of_variables, 3) * choose(3, 1) / choose(number_of_variables, 1) * first_level)

third_level = (choose(number_of_variables, 3) * mutual_info_profile[2] - 
               choose(number_of_variables, 3) * choose(3, 2) / choose(number_of_variables, 2) * second_level -
               choose(number_of_variables, 3) * choose(3, 1) / choose(number_of_variables, 1) * first_level)

fourth_level = (choose(number_of_variables, 4) * mutual_info_profile[3] - 
               choose(number_of_variables, 4) * choose(4, 3) / choose(number_of_variables, 3) * third_level -
               choose(number_of_variables, 4) * choose(4, 2) / choose(number_of_variables, 2) * second_level -
               choose(number_of_variables, 4) * choose(4, 1) * first_level)

print(second_level)
print(third_level)
print(fourth_level)

def calculate_information_levels(mutual_information_profile):
    information_levels = []
    number_of_variables = len(mutual_info_profile)
    for i in range(number_of_variables):
        print("i equals {}".format(i))
        information_level = choose(number_of_variables, i+1) * mutual_info_profile[i]
        j = i
        while j > 0: 
            information_level = information_level - (choose(number_of_variables, i+1) * choose(i+1, j) / 
                                choose(number_of_variables, j) * information_levels[j-1])
            j = j-1
            
        information_levels.append(information_level)
    
    return information_levels
        
print(calculate_information_levels(mutual_info_profile))

In [ ]:
marginal_distributions = np.array(
    [
        [0.3, 0.7],
        [0.4, 0.6],
        [0.6, 0.4],
        [0.6, 0.4]
    ]
)

print("now examing the copy function")
probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(marginal_distributions[2:])
)
pdf_extended6 = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
pdf_extended6.append_determinstic_function(copy, 2, [2, 2])
pdf_extended6.adjust_to_old_format()
prob_dict = ProbabilityDict(pdf_extended6.to_dict())
prob_dict.print_dict()
print(calculate_mi_profile(pdf_extended6, [2, 3], testing=False))


print("")
print("now for the xor function")
probabilities_arr = FullNestedArrayOfProbabilities(
    compute_joint_from_marginals(marginal_distributions[2:])
)
pdf_extended7 = JointProbabilityMatrixExtended(num_variables1, num_states1, probabilities_arr)
pdf_extended7.append_determinstic_function(xor, 1, [2])
pdf_extended7.adjust_to_old_format()
prob_dict = ProbabilityDict(pdf_extended7.to_dict())
prob_dict.print_dict()
print("the synergy in the distribution is {}".format(
    sum([prob_dict.calulate_mutual_information([i], [2]) for i in range(2)])
))

print(calculate_mi_profile(pdf_extended7, [2], testing=False))

